# Imports

In [1]:
import numpy as np
import pandas as pd
import random as rand

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D
from keras.models import Sequential

from sklearn.model_selection import train_test_split

2023-07-24 11:48:10.326802: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 11:48:11.780523: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 11:48:11.784628: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 11:48:14.941740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data importing and pre-processing

In [2]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 1, 64, 64)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [3]:
def generate_generator_input(no_of_samples):
    input = np.random.randn(100*no_of_samples)

    input = input.reshape(60,100)

    return input

In [4]:
def make_generator():
    model = Sequential()
    
    model.add(Dense(60*8*8, input_dim=100))
    model.add(Activation("relu"))
    model.add(Reshape((8,8,60)))
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(8,8,60)))
    model.add(Activation("relu"))
  
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(16,16,60)))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(32,32,60)))
    model.add(Activation("relu"))
              
    model.summary()
    
    return model
    
    

# Discriminator Model

In [5]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, kernel_size=5, padding="same", input_shape=(1,64,64)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    
    
    # output layer
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    model.add(Activation("sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model
    

# Training

First, create the models from the functions

In [6]:
gen_model = make_generator()
disc_model = make_discriminator()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3840)              387840    
                                                                 
 activation (Activation)     (None, 3840)              0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 60)          0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 16, 16, 1)         61        
 anspose)                                                        
                                                                 
 activation_1 (Activation)   (None, 16, 16, 1)         0         
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 32, 32, 1)         2         
 Transpose)                                             

Next, create fake training data, and put it together with the real data.

In [7]:
gen_input = generate_generator_input(60)

print(gen_input.shape)

x_fake = gen_model.predict(gen_input)
y_fake = np.ones((60,), dtype=int)

x_fake = x_fake.reshape(60, 1, 64, 64)
    
x_fake_train, x_fake_test = train_test_split(x_fake, test_size=0.2)
y_fake_train, y_fake_test = train_test_split(y_fake, test_size=0.2)

print("shapes:")
print("x_fake_train : " + str(x_fake_train.shape))
print("x_fake_test : " + str(x_fake_test.shape))
print("y_fake_train : " + str(y_fake_train.shape))
print("y_fake_test : " + str(y_fake_test.shape))

print("x_real_train : " + str(x_real_train.shape))
print("x_real_test : " + str(x_real_test.shape))
print("y_real_train : " + str(y_real_train.shape))
print("y_real_test : " + str(y_real_test.shape))

x_train = np.concatenate((x_fake_train, x_real_train))
y_train = np.concatenate((y_fake_train, y_real_train))
x_test = np.concatenate((x_fake_test, x_real_test))
y_test = np.concatenate((y_fake_test, y_real_test))

(60, 100)
2/2 [==============================] - 1s 13ms/step


2023-07-24 11:49:04.761492: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15698944 exceeds 10% of free system memory.
2023-07-24 11:49:04.763426: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15724544 exceeds 10% of free system memory.
2023-07-24 11:49:04.803298: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15698944 exceeds 10% of free system memory.
2023-07-24 11:49:04.804219: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15724544 exceeds 10% of free system memory.


shapes:
x_fake_train : (48, 1, 64, 64)
x_fake_test : (12, 1, 64, 64)
y_fake_train : (48,)
y_fake_test : (12,)
x_real_train : (48, 1, 64, 64)
x_real_test : (12, 1, 64, 64)
y_real_train : (48,)
y_real_test : (12,)


Now, train the discriminator

Disc training parameters:

In [8]:
N_EPOCHS = 12
BATCH_SIZE = 4
VERBOSE = 1


In [10]:
disc_model.compile(loss="binary_crossentropy")
disc_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=VERBOSE, validation_data=(x_test, y_test))

Epoch 1/12


ValueError: in user code:

    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 265, in __call__
        

Put models together into full GAN

In [ ]:
def make_GAN(gen,disc):
    disc.trainable = False

    model = Sequential()

    model.add(gen)
    model.add(disc)

    model.compile(loss='binary_crossentropy',optimizer='adam')

    model.summary()

    return model

Train GAN

In [ ]:
GAN_model = make_GAN(gen_model, disc_model)


# create some fake data as input

x_gan = generate_generator_input(60)
y_gan = np.ones((60))

GAN_model.fit(x_gan, y_gan)

Now that all training is done, generate an output value

In [ ]:
# first, create the input from noise
GAN_input = generate_generator_input(1)

GAN_output = GAN_model.predict(GAN_input)



Output result graphically